In [74]:
from functions import BECHMARK_FUNCTIONS, FUNCTION_RANGES
import numpy as np
import time
import optuna
from optuna.samplers import RandomSampler
from tqdm.auto import tqdm
import pandas as pd
from ray import tune
from ray.tune.search import BasicVariantGenerator
from hyperopt import fmin, tpe, hp, Trials

from metagen.framework import Domain
from metagen.metaheuristics import RandomSearch

optuna.logging.set_verbosity(optuna.logging.WARNING)

In [77]:
ITERATIONS = 1000

def optuna_objective(function_name: str, trial):
    x = np.array([trial.suggest_uniform(f'x{i}', FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1]) for i in range(2)])
    return BECHMARK_FUNCTIONS[function_name](x)

def run_optuna(function_name: str):

    start_time = time.time()
    study = optuna.create_study(direction='minimize', sampler=RandomSampler())
    study.optimize(lambda trial: optuna_objective(function_name, trial), n_trials=ITERATIONS, show_progress_bar=False)
    end_time = time.time()

    return {"library": "optuna", "function_name": function_name, "best_vale": study.best_value, "best_x0": study.best_params["x0"], "best_x1": study.best_params["x1"], "time": end_time - start_time}

def raytune_objective(config, function_name: str):
    x = np.array([config[f'x{i}'] for i in range(2)])

    return {"score": BECHMARK_FUNCTIONS[function_name](x)}

def run_raytune(function_name: str):
    search_space = {f'x{i}': tune.uniform(FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1]) for i in range(2)}
    
    start_time = time.time()
    analysis = tune.run(
        tune.with_parameters(raytune_objective, function_name=function_name),
        config=search_space,
        num_samples=ITERATIONS,
        search_alg=BasicVariantGenerator(),
        verbose=0,
        resources_per_trial={"cpu": 1}
    )
    end_time = time.time()
    
    best_trial = analysis.get_best_trial(metric="score", mode="min")
    best_config = best_trial.config
    best_loss = best_trial.metric_analysis["score"]["min"]
    
    return {"library": "raytune", "function_name": function_name, "best_vale": best_loss, "best_x0": best_config["x0"], "best_x1": best_config["x1"], "time": end_time - start_time}

def hyperopt_objective(params, function_name: str):
    x = np.array([params[f'x{i}'] for i in range(2)])
    return BECHMARK_FUNCTIONS[function_name](x)

def run_hyperopt(function_name: str):
    search_space = {f'x{i}': hp.uniform(f'x{i}', FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1]) for i in range(2)}
    
    trials = Trials()
    start_time = time.time()
    best = fmin(
        fn=lambda params: hyperopt_objective(params, function_name),
        space=search_space,
        algo=tpe.suggest,
        max_evals=ITERATIONS,
        trials=trials,
        show_progressbar=False
    )
    end_time = time.time()
    
    best_loss = min(trials.losses())
    
    return {"library": "hyperopt", "function_name": function_name, "best_vale": best_loss, "best_x0": best['x0'], "best_x1": best['x1'], "time": end_time - start_time}

def metagen_objective(solution, function_name: str):
    x = np.array([solution[f'x{i}'] for i in range(2)])
    return BECHMARK_FUNCTIONS[function_name](x)

def run_metagen(function_name: str):

    domain = Domain()
    
    domain.define_real(f'x0', FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1])
    domain.define_real(f'x1', FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1])

    start_time = time.time()
    search = RandomSearch(domain, lambda solution: metagen_objective(solution, function_name), population_size=10, max_iterations=ITERATIONS/10)
    best_solution = search.run()
    end_time = time.time()

    return {"library": "metagen", "function_name": function_name, "best_vale": best_solution.get_fitness(), "best_x0": best_solution["x0"], "best_x1": best_solution["x1"], "time": end_time - start_time}

In [78]:
results = []
for function_name in tqdm(BECHMARK_FUNCTIONS.keys()):
    solution_optuna = run_optuna(function_name)
    solution_hyperopt = run_hyperopt(function_name)
    solution_metagen = run_metagen(function_name)
     #solution_raytune = run_raytune(function_name)
    results.extend([solution_optuna, solution_hyperopt, solution_metagen]) #, solution_raytune])

#.to_csv("optuna_results.csv", index=False)

  0%|          | 0/9 [00:00<?, ?it/s]

C:\Users\manjj\AppData\Local\Temp\ipykernel_19568\2503941821.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  x = np.array([trial.suggest_uniform(f'x{i}', FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1]) for i in range(2)])


[0] F = 0.9985715740830193	{x0 = -0.2146560911381279 , x1 = 0.9759581633555401}
[1] F = 0.7694777836358573	{x0 = 0.6793339775394358 , x1 = 0.5549622785345925}
[2] F = 0.4826424372770405	{x0 = -0.11212279510704093 , x1 = 0.6856171789668233}
[3] F = 0.4826424372770405	{x0 = -0.11212279510704093 , x1 = 0.6856171789668233}
[4] F = 0.4826424372770405	{x0 = -0.11212279510704093 , x1 = 0.6856171789668233}
[5] F = 0.4826424372770405	{x0 = -0.11212279510704093 , x1 = 0.6856171789668233}
[6] F = 0.4826424372770405	{x0 = -0.11212279510704093 , x1 = 0.6856171789668233}
[7] F = 0.03428088219269651	{x0 = 0.027501409240739072 , x1 = -0.18309711817084917}
[8] F = 0.03428088219269651	{x0 = 0.027501409240739072 , x1 = -0.18309711817084917}
[9] F = 0.03428088219269651	{x0 = 0.027501409240739072 , x1 = -0.18309711817084917}
[10] F = 0.03428088219269651	{x0 = 0.027501409240739072 , x1 = -0.18309711817084917}
[11] F = 0.03428088219269651	{x0 = 0.027501409240739072 , x1 = -0.18309711817084917}
[12] F = 0.034

,library,function_name,best_vale,best_x0,best_x1,time
0,optuna,sphere,0.014134,0.013373,-0.118130,0.245531
1,hyperopt,sphere,0.000847,-0.020239,-0.020911,3.993834
2,metagen,sphere,0.004453,-0.059528,0.030155,0.024840
3,optuna,rastrigin,0.589341,-0.037866,-0.039387,0.247290
4,hyperopt,rastrigin,0.992441,-0.069128,0.017271,3.975890
5,metagen,rastrigin,4.699003,-2.044400,-0.025904,0.029818
6,optuna,rosenbrock,0.015492,1.118928,1.248328,0.250196
7,hyperopt,rosenbrock,0.004538,1.034602,1.064621,3.795627
8,metagen,rosenbrock,0.094190,1.266024,1.618120,0.031423
9,optuna,ackley,3.412625,-0.113242,-0.643295,0.257617


In [84]:
total_results = pd.DataFrame(results)
total_results.groupby([ "function_name", "library"]).first().to_excel("results.xlsx")